*This notebook gets the raw atlas traceroute measurement data from `ra_traceroute_parameters` table; prunes the data to fetch (probeid, timestamp, h1_latencies, h2_latencies) then inserts entire dataframe in `ra_latencies` table.*
- - - 

In [58]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [59]:
DB_LOCATION = 'lastmile.db'
RA_TRACEROUTE_PARAMETERS = 'ra_traceroute_parameters'
RA_LATENCIES_LAST_HOP = 'ra_latencies_last_hop'

In [60]:
con = sqlite3.connect(DB_LOCATION)

In [61]:
query = '''SELECT    probeid
                   , starttime as timestamp
                   , GROUP_CONCAT(result_hop_no, ';') as hop_no
                   , GROUP_CONCAT(result_hop_blob, ';') as result_hop_blob
                   , '' as last_hop
                   , '' as last_hop_blob                   
                   , '' as last_hop_latencies
           FROM      %s
           GROUP BY  probeid, starttime
        '''%(RA_TRACEROUTE_PARAMETERS)

df = pd.read_sql(query, con)

In [62]:
def get_last_hop(hop_no): last_hop = hop_no.split(';')[-1]; return last_hop
def get_last_hop_blob(result_hop_blob): last_hop_blob = result_hop_blob.split(';')[-1]; return last_hop_blob

df['last_hop'] = df['hop_no'].apply(get_last_hop)
df['last_hop_blob'] = df['result_hop_blob'].apply(get_last_hop_blob)

In [63]:
df.head()

,probeid,timestamp,hop_no,result_hop_blob,last_hop,last_hop_blob,last_hop_latencies
0,10006,1406548034,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18,"[{'size': 76, 'rtt': 0.559, 'ttl': 64, 'from':...",18,"[{'size': 48, 'rtt': 64.929, 'ttl': 55, 'from'...",
1,10006,1406562432,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18,"[{'size': 76, 'rtt': 0.579, 'ttl': 64, 'from':...",18,"[{'size': 48, 'rtt': 64.001, 'ttl': 55, 'from'...",
2,10006,1406576835,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18,"[{'size': 76, 'rtt': 0.58, 'ttl': 64, 'from': ...",18,"[{'size': 48, 'rtt': 64.473, 'ttl': 55, 'from'...",
3,10006,1406591230,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18,"[{'size': 76, 'rtt': 0.769, 'ttl': 64, 'from':...",18,"[{'size': 48, 'rtt': 64.46, 'ttl': 55, 'from':...",
4,10006,1406605631,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18,"[{'size': 76, 'rtt': 0.534, 'ttl': 64, 'from':...",18,"[{'size': 48, 'rtt': 64.059, 'ttl': 55, 'from'...",


In [64]:
del df['hop_no']
del df['result_hop_blob']

In [65]:
df.head()

,probeid,timestamp,last_hop,last_hop_blob,last_hop_latencies
0,10006,1406548034,18,"[{'size': 48, 'rtt': 64.929, 'ttl': 55, 'from'...",
1,10006,1406562432,18,"[{'size': 48, 'rtt': 64.001, 'ttl': 55, 'from'...",
2,10006,1406576835,18,"[{'size': 48, 'rtt': 64.473, 'ttl': 55, 'from'...",
3,10006,1406591230,18,"[{'size': 48, 'rtt': 64.46, 'ttl': 55, 'from':...",
4,10006,1406605631,18,"[{'size': 48, 'rtt': 64.059, 'ttl': 55, 'from'...",


In [66]:
def return_all_rtt_field_values(json_string):
    import json
    try: json_data = json.loads(json_string)
    except Exception as e: print(e, type(e), json_string)
    try: 
        rtt_field_values = ', '.join([str(item['rtt']) for item in json_data])        
    except Exception as e: print('*', end=""); return None
    else: return rtt_field_values

In [67]:
def return_concat_from_field_values(a):
    a = a.replace("'", '"')
    a = return_all_rtt_field_values(a)
    return a

In [68]:
def get_rtts(last_hop_blob):
    last_hop_blob = last_hop_blob.replace('True', '"True"')
    rtts = return_concat_from_field_values(last_hop_blob)
    return rtts    
df['last_hop_latencies'] = df['last_hop_blob'].apply(get_rtts)

***********************************************************************************************************************************************************************************************************************Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
**************************Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
*Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
***************************************************************************Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
****************************************************************Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
************************************************************************************Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
**********Expecting value: line 1 column 1 (char 0) <class 'ValueError'> nan
*Expecting value: line 1 column 1 (char 0) <class 'Value

In [69]:
df.head()

,probeid,timestamp,last_hop,last_hop_blob,last_hop_latencies
0,10006,1406548034,18,"[{'size': 48, 'rtt': 64.929, 'ttl': 55, 'from'...","64.929, 65.337, 63.992"
1,10006,1406562432,18,"[{'size': 48, 'rtt': 64.001, 'ttl': 55, 'from'...","64.001, 64.404, 64.194"
2,10006,1406576835,18,"[{'size': 48, 'rtt': 64.473, 'ttl': 55, 'from'...","64.473, 64.215, 64.613"
3,10006,1406591230,18,"[{'size': 48, 'rtt': 64.46, 'ttl': 55, 'from':...","64.46, 65.16, 63.902"
4,10006,1406605631,18,"[{'size': 48, 'rtt': 64.059, 'ttl': 55, 'from'...","64.059, 63.728, 64.401"


In [70]:
del df['last_hop_blob']

In [71]:
df.head()

,probeid,timestamp,last_hop,last_hop_latencies
0,10006,1406548034,18,"64.929, 65.337, 63.992"
1,10006,1406562432,18,"64.001, 64.404, 64.194"
2,10006,1406576835,18,"64.473, 64.215, 64.613"
3,10006,1406591230,18,"64.46, 65.16, 63.902"
4,10006,1406605631,18,"64.059, 63.728, 64.401"


In [72]:
df.count()

probeid               137017
timestamp             137017
last_hop              137017
last_hop_latencies    135815
dtype: int64

In [73]:
cur = con.execute('pragma foreign_keys=ON')

In [74]:
index_label = ['probeid', 'timestamp']
df = df.set_index(index_label)

In [75]:
df.head()

last_hop      last_hop_latencies
probeid timestamp                                  
10006   1406548034       18  64.929, 65.337, 63.992
        1406562432       18  64.001, 64.404, 64.194
        1406576835       18  64.473, 64.215, 64.613
        1406591230       18    64.46, 65.16, 63.902
        1406605631       18  64.059, 63.728, 64.401

In [76]:
df.to_sql(  '%s'%RA_LATENCIES_LAST_HOP
          , con
          , flavor='sqlite'
          , if_exists = 'append'
          , index_label = index_label
         )

In [77]:
con.commit()
con.close()